In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

## Pivoting Tables

There are a number of basic operations for rearranging tabular data. These are alter‐ natingly referred to as reshape or pivot operations.

- `stack()` pivots from the columns in the data into rows
- `unstack()` pivots from the rows in the data into columns

**Pivoting "Long" to "Wide" Format**

- `pivot(row,column,value)`
    - `row` refers to the variable used as the row index of the pivot table
    - `column` refers to the variable used as the column index of the pivot table
    - `value` refers to the variable used to fill the DataFrame
    
**pivoting "wide" to "Long" Format**

- `pd.melt(df,id_var=[],value_vars=[])`
    - `id_vars` refers to the variable used as the row labels
    - `value_vars` refers to varaibles used as value columns

### Question 1

Write an SQL query to reformat the table such that there is a department id column and a revenue column for each month.

`Department`

| id   | revenue | month |
|------|---------|-------|
| 1    | 8000    | Jan   |
| 2    | 9000    | Jan   |
| 3    | 10000   | Feb   |
| 1    | 7000    | Feb   |
| 1    | 6000    | Mar   |



Note that the result table has 13 columns (1 for the department id + 12 for the months).


In [2]:
Department = pd.DataFrame({"id":[1,2,3,1,1],
             "revenue":[8000,9000,10000,7000,6000],
             "month":["Jan","Jan","Feb","Feb",'Mar']})
Department

,id,revenue,month
0,1,8000,Jan
1,2,9000,Jan
2,3,10000,Feb
3,1,7000,Feb
4,1,6000,Mar


The question requires us to convert each row of month into columns. This is a typical question of pivoting "Long" table to "wide" table.

In [4]:
pd.pivot(Department,"id","month","revenue")

month,Feb,Jan,Mar
id,,,
1,7000.0,8000.0,6000.0
2,NaN,9000.0,NaN
3,10000.0,NaN,NaN


As we can see there are only three columns for month, but we need 12 columns representing 12 months. So we need to construct months from Jan to Dec.

In [73]:
from datetime import datetime

months = pd.DataFrame({"month":list(range(1,13))})
abbre_months = months["month"].apply(lambda month:datetime.strftime(datetime.strptime(str(month),"%m"),"%b"),)
months["month"] = abbre_months

# merge the month table with the dept table on months
merged_df = pd.merge(Department,abbre_months,on="month",how="right")

# pivot from the rows in the table into columns
pivoted_df = pd.pivot(merged_df,"id","month","revenue").iloc[1:,:]

# Reindexing changes the row labels and column labels of a DataFrame. 
# To reindex means to conform the data to match a given set of labels along a particular axis. 
# Reorder the existing data to match a new set of labels.
pivoted_df_ = pivoted_df.reindex(abbre_months.values,axis=1)
pivoted_df.columns = pivoted_df_.columns.map(lambda x:x+"_Revenue")
pivoted_df


month,Jan_Revenue,Feb_Revenue,Mar_Revenue,Apr_Revenue,May_Revenue,Jun_Revenue,Jul_Revenue,Aug_Revenue,Sep_Revenue,Oct_Revenue,Nov_Revenue,Dec_Revenue
id,,,,,,,,,,,,
1.0,NaN,NaN,NaN,7000.0,8000.0,NaN,NaN,6000.0,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Question 2: Create a Session Bar Chart

You want to know how long a user visits your application. You decided to create bins of "[0-5>", "[5-10>", "[10-15>" and "15 minutes or more" and count the number of sessions on it.

Write an SQL query to report the (bin, total) in any order.

The query result format is in the following example.

`Sessions`

| session_id  | duration      |
|-------------|---------------|
| 1           | 30            |
| 2           | 199           |
| 3           | 299           |
| 4           | 580           |
| 5           | 1000          |


`result`

| bin          | total        |
|--------------|--------------|
| [0-5>        | 3            |
| [5-10>       | 1            |
| [10-15>      | 0            |
| 15 or more   | 1            |



In [75]:
Sessions = pd.DataFrame({"session_id":[1,2,3,4,5],
             "duration":[30,199,299,580,1000]})
Sessions

,session_id,duration
0,1,30
1,2,199
2,3,299
3,4,580
4,5,1000


In [77]:
def bin(x):
    if x>=0 and x<5:
        return "[0-5>"
    elif x>=5 and x<10:
        return "[5-10>"
    elif x>=10 and x<15:
        return "[10-15>"
    else:
        return "15 minutes or more"

Sessions["bins"] = Sessions["duration"].apply(lambda x:bin(x/60))
Sessions

,session_id,duration,bins
0,1,30,[0-5>
1,2,199,[0-5>
2,3,299,[0-5>
3,4,580,[5-10>
4,5,1000,15 minutes or more


In [93]:
bin_df = pd.DataFrame({"bins":["[0-5>","[5-10>","[10-15>","15 minutes or more"]})
result = pd.merge(bin_df,Sessions,on="bins",how='left').groupby("bins").\
agg({"session_id":"count"}).rename(columns={"session_id":"total"}).\
reindex(bin_df["bins"])
result


,total
bins,
[0-5>,3
[5-10>,1
[10-15>,0
15 minutes or more,1


### Question 3: Product's Price for Each Store

Write an SQL query to find the price of each product in each store.

Return the result table in any order.

The query result format is in the following example:

`Products`

| product_id  | store  | price |
|-------------|--------|-------|
| 0           | store1 | 95    |
| 0           | store3 | 105   |
| 0           | store2 | 100   |
| 1           | store1 | 70    |
| 1           | store3 | 80    |

`result`

| product_id  | store1 | store2 | store3 |
|-------------|--------|--------|--------|
| 0           | 95     | 100    | 105    |
| 1           | 70     | null   | 80     |


**Pivoting a long table to a wide format**

we can use pivot function

In [33]:
Sales = pd.DataFrame({"product_id":[0,0,0,1,1],
             "store":["store1","store3","store2","store1","store3"],
             "price":[95,105,100,70,80]})
Sales

,product_id,store,price
0,0,store1,95
1,0,store3,105
2,0,store2,100
3,1,store1,70
4,1,store3,80


In [101]:
pd.pivot(Products,"product_id","store","price")

store,store1,store2,store3
product_id,,,
0,95.0,100.0,105.0
1,70.0,NaN,80.0


In [35]:
Sales.groupby("product_id").mean()

,price
product_id,
0,100.0
1,75.0


### Question 4: Rearrange Products Table

Write an SQL query to rearrange the Products table so that each row has (product_id, store, price). If a product is not available in a store, do not include a row with that product_id and store combination in the result table.

Return the result table in any order.

The query result format is in the following example:

`Products`

| product_id | store1 | store2 | store3 |
|------------|--------|--------|--------|
| 0          | 95     | 100    | 105    |
| 1          | 70     | null   | 80     |

`Result`

| product_id | store  | price |
|------------|--------|-------|
| 0          | store1 | 95    |
| 0          | store2 | 100   |
| 0          | store3 | 105   |
| 1          | store1 | 70    |
| 1          | store3 | 80    |


In [3]:
Products = pd.DataFrame({"product_id":[0,1],
             "store1":[95,70],
             "store2":[100,np.nan],
             "store3":[105,80]})
Products

,product_id,store1,store2,store3
0,0,95,100.0,105
1,1,70,NaN,80


**This is a typical question of pivoting columns in the data into rows. In other words, we need to pivot a wide format to long format**

In [11]:
melted_df = pd.melt(Products,id_vars=["product_id"],
                    value_vars=["store1","store2","store3"],
                    var_name="store",value_name="price",
                   ignore_index=True)
result = melted_df[melted_df["price"].notnull()].sort_values(by="product_id")
result

,product_id,store,price
0,0,store1,95.0
2,0,store2,100.0
4,0,store3,105.0
1,1,store1,70.0
5,1,store3,80.0


### Question 5: User Purchase Platform

Write an SQL query to find the total number of users and the total amount spent using mobile only, desktop only and both mobile and desktop together for each date.

The query result format is in the following example:

`Spending`

| user_id | spend_date | platform | amount |
|---------|------------|----------|--------|
| 1       | 2019-07-01 | mobile   | 100    |
| 1       | 2019-07-01 | desktop  | 100    |
| 2       | 2019-07-01 | mobile   | 100    |
| 2       | 2019-07-02 | mobile   | 100    |
| 3       | 2019-07-01 | desktop  | 100    |
| 3       | 2019-07-02 | desktop  | 100    |



In [4]:
Spending = pd.DataFrame({"user_id":[1,1,2,2,3,3],
             "spend_date":["2019-07-01"]*3+["2019-07-02","2019-07-01","2019-07-02"],
             "platform":["mobile","desktop","mobile","mobile","desktop","desktop"],
             "amount":[100]*6})
Spending

,user_id,spend_date,platform,amount
0,1,2019-07-01,mobile,100
1,1,2019-07-01,desktop,100
2,2,2019-07-01,mobile,100
3,2,2019-07-02,mobile,100
4,3,2019-07-01,desktop,100
5,3,2019-07-02,desktop,100


In [5]:
platforms = {}
for date in Spending['spend_date']:
    platforms[date] = ['mobile','desktop',"both"]
platform_df = pd.melt(pd.DataFrame(platforms),var_name="spend_date",value_name="platform")
platform_df

,spend_date,platform
0,2019-07-01,mobile
1,2019-07-01,desktop
2,2019-07-01,both
3,2019-07-02,mobile
4,2019-07-02,desktop
5,2019-07-02,both


In [15]:
def p_type(x):
    if len(set(x))==1:
        return x[0]
    elif len(set(x))>1:
        return "both"

res1 = Spending.groupby(["spend_date","user_id"]).apply(lambda x:p_type(x["platform"].values)).to_frame().\
rename(columns={0:"platform"}).reset_index()
res2 = Spending.groupby(["spend_date","user_id"]).agg({"amount":"sum"}).reset_index()
res = pd.merge(res1,res2)

result = pd.merge(platform_df,res,on=["spend_date","platform"],how="left").groupby(["spend_date","platform"]).\
agg({"user_id":"count","amount":"sum"})
result.reset_index()

,spend_date,platform,user_id,amount
0,2019-07-01,both,1,200.0
1,2019-07-01,desktop,1,100.0
2,2019-07-01,mobile,1,100.0
3,2019-07-02,both,0,0.0
4,2019-07-02,desktop,1,100.0
5,2019-07-02,mobile,1,100.0


### Question 6: Number of Transactions per Visit

A bank wants to draw a chart of the number of transactions bank visitors did in one visit to the bank and the corresponding number of visitors who have done this number of transaction in one visit.

Write an SQL query to find how many users visited the bank and didn't do any transactions, how many visited the bank and did one transaction and so on.

The result table will contain two columns:

transactions_count which is the number of transactions done in one visit.
visits_count which is the corresponding number of users who did transactions_count in one visit to the bank.

transactions_count should take all values from 0 to max(transactions_count) done by one or more users.

Order the result table by transactions_count.

The query result format is in the following example:

`Visits`

| user_id | visit_date |
|---------|------------|
| 1       | 2020-01-01 |
| 2       | 2020-01-02 |
| 12      | 2020-01-01 |
| 19      | 2020-01-03 |
| 1       | 2020-01-02 |
| 2       | 2020-01-03 |
| 1       | 2020-01-04 |
| 7       | 2020-01-11 |
| 9       | 2020-01-25 |
| 8       | 2020-01-28 |


`Transaction`

| user_id | transaction_date | amount |
|---------|------------------|--------|
| 1       | 2020-01-02       | 120    |
| 2       | 2020-01-03       | 22     |
| 7       | 2020-01-11       | 232    |
| 1       | 2020-01-04       | 7      |
| 9       | 2020-01-25       | 33     |
| 9       | 2020-01-25       | 66     |
| 8       | 2020-01-28       | 1      |
| 9       | 2020-01-25       | 99     |



In [214]:
Visits = pd.DataFrame({"user_id":[1,2,12,19,1,2,1,7,9,8],
             "visit_date":["2020-01-01","2020-01-02","2020-01-01","2020-01-03","2020-01-02","2020-01-03",
                           "2020-01-04","2020-01-11","2020-01-25","2020-01-28"]})
Transactions = pd.DataFrame({"user_id":[1,2,7,1,9,9,8,9],
                            "transaction_date":["2020-01-02","2020-01-03","2020-01-11","2020-01-04","2020-01-25",
                                               "2020-01-25","2020-01-28","2020-01-25"],
                            "amount":[120,22,232,7,33,66,1,99]})

We can compute the number of transactions that each visitor conducted in one visit date. Considering that not every visitor conducted transactions in one visit, we can expect that there are zero transactions in certain visit dates. But we need to display all visit dates even if the visit date does not include any transaction.

In [227]:
merged_df = pd.merge(Visits,Transactions,left_on=["user_id","visit_date"],right_on=["user_id","transaction_date"],
        how="left")
transaction_df = merged_df.groupby(["user_id","visit_date"]).agg({"transaction_date":"count"}).rename(columns={"transaction_date":"transaction_count"}).reset_index()
max_transactions = transaction_df["transaction_count"].max()
transaction_df

,user_id,visit_date,transaction_count
0,1,2020-01-01,0
1,1,2020-01-02,1
2,1,2020-01-04,1
3,2,2020-01-02,0
4,2,2020-01-03,1
5,7,2020-01-11,1
6,8,2020-01-28,1
7,9,2020-01-25,3
8,12,2020-01-01,0
9,19,2020-01-03,0


In [233]:
# construct a dataframe with all possible transactions
possible_transactions = pd.DataFrame({"transaction_count":list(range(0,max_transactions+1))})

# merge the two tables
merged_trans = pd.merge(transaction_df,possible_transactions,on="transaction_count",how='right')

# group the table by trans count
res = merged_trans.groupby("transaction_count").agg({"user_id":"count"})
res

,user_id
transaction_count,
0,4
1,5
2,0
3,1


### Question 7: Total Sales Amount by Year

Write an SQL query to report the Total sales amount of each item for each year, with corresponding product name, product_id, report_year, and total_amount.

Dates of the sales years are between 2018 to 2020. Return the result table ordered by product_id and report_year.

The query result format is in the following example:


`Product`

| product_id | product_name |
|------------|--------------|
| 1          | LC Phone     |
| 2          | LC T-Shirt   |
| 3          | LC Keychain  |

`Sales`

| product_id | period_start | period_end  | average_daily_sales |
|------------|--------------|-------------|---------------------|
| 1          | 2019-01-25   | 2019-02-28  | 100                 |
| 2          | 2018-12-01   | 2020-01-01  | 10                  |
| 3          | 2019-12-01   | 2020-01-31  | 1                   |



In [235]:
Product = pd.DataFrame({"product_id":[1,2,3],
                       "product_name":["LC Phone","LC T-Shirt","LC Keychain"]})
Sales = pd.DataFrame({"product_id":[1,2,3],
             "period_start":["2019-01-25","2018-12-01","2019-12-01"],
             "period_end":["2019-02-28","2020-01-01","2020-01-31"],
             "average_daily_sales":[100,10,1]})

In [240]:
sales = Sales.copy()
sales["period_start"] = sales["period_start"].map(lambda x:datetime.strptime(x,"%Y-%m-%d"))
sales["period_end"] = sales["period_end"].map(lambda x:datetime.strptime(x,"%Y-%m-%d"))
sales["days"] = sales["period_end"] - sales["period_start"]
sales

,product_id,period_start,period_end,average_daily_sales,days
0,1,2019-01-25,2019-02-28,100,34 days
1,2,2018-12-01,2020-01-01,10,396 days
2,3,2019-12-01,2020-01-31,1,61 days


In [316]:
# then we find the longest timer period
from pandas.tseries.offsets import Hour, Minute, Day

days_ser = sales["days"].apply(lambda x:np.arange(0,x.days+1))
days_ser.index = np.array([1,2,3])
days = []
user_ids = []
k = 1
for i in range(1,len(days_ser)+1):
    days.extend(days_ser[i])
    days_len = len(days_ser[i])
    user_ids.extend([k for day in range(days_len)])
    k += 1

In [340]:
df1 = pd.DataFrame({"product_id":user_ids,"days":days})
merged_df = pd.merge(df1,sales,on="product_id",suffixes=("_diff","_periodDiff"))
merged_df["day_offset"] = merged_df["days_diff"].map(lambda x:Day(x))
merged_df["report_year"] = (merged_df["period_start"] + merged_df["day_offset"]).apply(lambda x:x.year)
merged_df.groupby(["product_id","report_year"]).agg({"average_daily_sales":"sum"}).reset_index().\
merge(Product,on="product_id").reindex(["product_id","product_name","report_year","average_daily_sales"],axis=1)

/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimelike.py:1187: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  warnings.warn(


,product_id,product_name,report_year,average_daily_sales
0,1,LC Phone,2019,3500
1,2,LC T-Shirt,2018,310
2,2,LC T-Shirt,2019,3650
3,2,LC T-Shirt,2020,10
4,3,LC Keychain,2019,31
5,3,LC Keychain,2020,31


### Question 8: Find the Missing IDs

Write an SQL query to find the missing customer IDs. The missing IDs are ones that are not in the Customers table but are in the range between 1 and the maximum customer_id present in the table.

Notice that the maximum customer_id will not exceed 100.

Return the result table ordered by ids in ascending order.

The query result format is in the following example.

`Customers`

| customer_id | customer_name |
|-------------|---------------|
| 1           | Alice         |
| 4           | Bob           |
| 5           | Charlie       |



In [343]:
Customers = pd.DataFrame({"customer_id":[1,4,5],
             "customer_name":["Alice","Bob","Charlie"]})
Customers

,customer_id,customer_name
0,1,Alice
1,4,Bob
2,5,Charlie


The missing customer ids are those not appearing in the customer table. So we firt need to create one table containing all customers.

In [349]:
full_customers = pd.DataFrame({"id":range(1,Customers["customer_id"].max()+1)})
full_customers[~full_customers["id"].isin(Customers["customer_id"])]

,id
1,2
2,3


## Question 9: Hopper Company Queries I

Write an SQL query to report the following statistics for each month of 2020:

- The number of drivers currently with the Hopper company by the end of the month (active_drivers).

- The number of accepted rides in that month (accepted_rides).

Return the result table ordered by month in ascending order, where month is the month's number (January is 1, February is 2, etc.).

The query result format is in the following example.

`Drivers`

| driver_id | join_date  |
|-----------|------------|
| 10        | 2019-12-10 |
| 8         | 2020-1-13  |
| 5         | 2020-2-16  |
| 7         | 2020-3-8   |
| 4         | 2020-5-17  |
| 1         | 2020-10-24 |
| 6         | 2021-1-5   |

`Rides`

| ride_id | user_id | requested_at |
|---------|---------|--------------|
| 6       | 75      | 2019-12-9    |
| 1       | 54      | 2020-2-9     |
| 10      | 63      | 2020-3-4     |
| 19      | 39      | 2020-4-6     |
| 3       | 41      | 2020-6-3     |
| 13      | 52      | 2020-6-22    |
| 7       | 69      | 2020-7-16    |
| 17      | 70      | 2020-8-25    |
| 20      | 81      | 2020-11-2    |
| 5       | 57      | 2020-11-9    |
| 2       | 42      | 2020-12-9    |
| 11      | 68      | 2021-1-11    |
| 15      | 32      | 2021-1-17    |
| 12      | 11      | 2021-1-19    |
| 14      | 18      | 2021-1-27    |

`AcceptedRides`

| ride_id | driver_id | ride_distance | ride_duration |
|---------|-----------|---------------|---------------|
| 10      | 10        | 63            | 38            |
| 13      | 10        | 73            | 96            |
| 7       | 8         | 100           | 28            |
| 17      | 7         | 119           | 68            |
| 20      | 1         | 121           | 92            |
| 5       | 7         | 42            | 101           |
| 2       | 4         | 6             | 38            |
| 11      | 8         | 37            | 43            |
| 15      | 8         | 108           | 82            |
| 12      | 8         | 38            | 34            |
| 14      | 1         | 90            | 74            |


`result`

| month | active_drivers | accepted_rides |
|-------|----------------|----------------|
| 1     | 2              | 0              |
| 2     | 3              | 0              |
| 3     | 4              | 1              |
| 4     | 4              | 0              |
| 5     | 5              | 0              |
| 6     | 5              | 1              |
| 7     | 5              | 1              |
| 8     | 5              | 1              |
| 9     | 5              | 0              |
| 10    | 6              | 0              |
| 11    | 6              | 2              |
| 12    | 6              | 1              |



In [359]:
dates = ["2019-12-10","2020-01-13","2020-02-16","2020-03-08","2020-05-17","2020-10-24","2021-01-05"]
Drivers = pd.DataFrame({"driver_id":[10,9,5,7,4,1,6],
             "join_date":pd.to_datetime(dates)})

dates_1 = ["2019-12-09","2020-02-09","2020-03-04","2020-04-06","2020-06-03",
          "2020-06-22","2020-07-16","2020-08-25","2020-11-02","2020-11-09","2020-12-09",
          "2021-01-11","2021-01-17","2021-01-19","2021-01-27"]
Rides = pd.DataFrame({"ride_id":[6,1,10,19,3,13,7,17,20,5,2,11,15,12,14],
                     "user_id":[75,54,63,39,41,52,69,70,81,57,42,68,32,11,18],
                     "requested_at":pd.to_datetime(dates_1)})

AcceptedRides = pd.DataFrame({"ride_id":[10,13,7,17,20,5,2,11,15,12,14],
                             "driver_id":[10,10,8,7,1,7,4,8,8,8,1],
                             "ride_distance":[63,73,100,119,121,42,6,37,108,38,90],
                             "ride_duration":[38,96,28,68,92,101,38,43,82,34,74]})

*Note:we need to compute the statistics for each month of 2020, meaning that we need one table contains months from January to December*

In [361]:
# first we create one dataframe containing each month fo 2020
months = pd.DataFrame({"month":np.arange(1,13)})
months

,month
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


**Compute the first statistic: the number of active drivers by the end of each month**

For example, by the end of January of 2020, there are two active drivers 10 and 8. We can compute the number of drivers based on the logic.

In [389]:
# copy the original dataframe
drivers = Drivers.copy()
# filter data not in the year of 2020
drivers = drivers[drivers["join_date"] < datetime(2021,1,1)]

# then convert the december of 2019 to January of 2020
def convert_month(x):
    if x.year < 2020:
        return 1
    else:
        return x.month
drivers["month_int"] = drivers["join_date"].apply(lambda x:convert_month(x))
merged_drivers = pd.merge(drivers,months,left_on="month_int",right_on="month",how='right')
drivers_num = merged_drivers.groupby("month").agg({"driver_id":"count"}).cumsum().\
rename(columns={"driver_id":"active_drivers"}).reset_index()
drivers_num

,month,active_drivers
0,1,2
1,2,3
2,3,4
3,4,4
4,5,5
5,6,5
6,7,5
7,8,5
8,9,5
9,10,6


**Compute the second statistics: the number of accepted rides**

In [402]:
rides = Rides[Rides["ride_id"].isin(AcceptedRides["ride_id"])]
rides_20 = rides[rides["requested_at"] < datetime(2021,1,1)]
rides_20["month"] = rides_20["requested_at"].map(lambda x:x.month)
rides_20 = pd.merge(months,rides_20,on="month",how="left").groupby("month").agg({"requested_at":"count"}).\
reset_index().rename(columns={"requested_at":"accepted_rides"})
rides_20

/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,month,accepted_rides
0,1,0
1,2,0
2,3,1
3,4,0
4,5,0
5,6,1
6,7,1
7,8,1
8,9,0
9,10,0


In [403]:
pd.merge(drivers_num,rides_20,on="month")

,month,active_drivers,accepted_rides
0,1,2,0
1,2,3,0
2,3,4,1
3,4,4,0
4,5,5,0
5,6,5,1
6,7,5,1
7,8,5,1
8,9,5,0
9,10,6,0


### Question 10: Find the Subtasks That Did Not Execute

Write an SQL query to report the IDs of the missing subtasks for each task_id.

Return the result table in any order.

The query result format is in the following example:


`Tasks`

| task_id | subtasks_count |
|---------|----------------|
| 1       | 3              |
| 2       | 2              |
| 3       | 4              |

task_id is the primary key for this table.
Each row in this table indicates that task_id was divided into subtasks_count subtasks labelled from 1 to subtasks_count.
It is guaranteed that 2 $\le$ subtasks_count $\le$ 20.


`Executed`

| task_id | subtask_id |
|---------|------------|
| 1       | 2          |
| 3       | 1          |
| 3       | 2          |
| 3       | 3          |
| 3       | 4          |

`result`

| task_id | subtask_id |
|---------|------------|
| 1       | 1          |
| 1       | 3          |
| 2       | 1          |
| 2       | 2          |



In [405]:
Tasks = pd.DataFrame({"task_id":[1,2,3],i 
                     "subtasks_count":[3,2,4]})

Executed = pd.DataFrame({"task_id":[1,3,3,3,3],
             "subtask_id":[2,1,2,3,4]})

**We need to create a table that create all subtasks for each task**

In [463]:
def compute_subtasks(x):
    """
    compute the index and subtasks
    """ 
    index = tasks[tasks["task_id"] == x].index[0]
    return [x]*tasks["subtasks_count"][index]
tasks_id = tasks["task_id"].apply(lambda x:compute_subtasks(x))
tasks_id_list = []
for i in range(len(tasks_id)):
    tasks_id_list += tasks_id[i]

# list all subtasks for each task
subtasks = tasks["subtasks_count"].map(lambda x:[i for i in range(1,x+1)])
subtasks_list = []
for i in range(len(subtasks)):
    subtasks_list += subtasks[i]
    
possible_subtasks = pd.DataFrame({"task_id":tasks_id_list,"subtask_id":subtasks_list})

In [472]:
Executed["combined_task"] = Executed["task_id"].map(lambda x:str(x)) + \
Executed["subtask_id"].map(lambda x:str(x))
Executed

,task_id,subtask_id,combined_task
0,1,2,12
1,3,1,31
2,3,2,32
3,3,3,33
4,3,4,34


In [479]:
possible_subtasks["combined_task"] = possible_subtasks["task_id"].map(lambda x:str(x)) +\
possible_subtasks["subtask_id"].map(lambda x:str(x))
result = possible_subtasks[~possible_subtasks["combined_task"].isin(Executed["combined_task"])].drop("combined_task",axis=1)
result

,task_id,subtask_id
0,1,1
2,1,3
3,2,1
4,2,2


### Question 11: Students Report By Geography

A U.S graduate school has students from Asia, Europe and America. The students' location information are stored in table student as below.

| name   | continent |
|--------|-----------|
| Jack   | America   |
| Pascal | Europe    |
| Xi     | Asia      |
| Jane   | America   |

Pivot the continent column in this table so that each name is sorted alphabetically and displayed underneath its corresponding continent. The output headers should be America, Asia and Europe respectively. It is guaranteed that the student number from America is no less than either Asia or Europe

| America | Asia | Europe |
|---------|------|--------|
| Jack    | Xi   | Pascal |
| Jane    |      |        |



In [12]:
Student = pd.DataFrame({"name":["Jack","Pascal","Xi","Jane"],
                        "continent":["America","Europe","Asia","America"]})
Student

,name,continent
0,Jack,America
1,Pascal,Europe
2,Xi,Asia
3,Jane,America


In [13]:
student = Student.reset_index().copy()
student["rank"] = student["name"].groupby(student["continent"]).rank(ascending=True,method="first")
result = student.pivot_table(
    index="rank",
    columns="continent",
    values="name",
    aggfunc="max",
    fill_value="")
result

continent,America,Asia,Europe
rank,,,
1.0,Jack,Xi,Pascal
2.0,Jane,,


In [31]:
res = result.reset_index().drop("rank",axis=1)
res

continent,America,Asia,Europe
0,Jack,Xi,Pascal
1,Jane,,


## Question: [Sales by Day of the Week](https://leetcode-cn.com/problems/sales-by-day-of-the-week/)

`Orders`

| order_id   | customer_id  | order_date  | item_id      | quantity    |
|------------|--------------|-------------|--------------|-------------|
| 1          | 1            | 2020-06-01  | 1            | 10          |
| 2          | 1            | 2020-06-08  | 2            | 10          |
| 3          | 2            | 2020-06-02  | 1            | 5           |
| 4          | 3            | 2020-06-03  | 3            | 5           |
| 5          | 4            | 2020-06-04  | 4            | 1           |
| 6          | 4            | 2020-06-05  | 5            | 5           |
| 7          | 5            | 2020-06-05  | 1            | 10          |
| 8          | 5            | 2020-06-14  | 4            | 5           |
| 9          | 5            | 2020-06-21  | 3            | 5           |

`items`

| item_id    | item_name      | item_category |
|------------|----------------|---------------|
| 1          | LC Alg. Book   | Book          |
| 2          | LC DB. Book    | Book          |
| 3          | LC SmarthPhone | Phone         |
| 4          | LC Phone 2020  | Phone         |
| 5          | LC SmartGlass  | Glasses       |
| 6          | LC T-Shirt XL  | T-Shirt       |

You are the business owner and would like to obtain a sales report for category items and day of the week. Write an SQL query to report how many units in each category have been ordered on each day of the week. Return the result table ordered by category. The query result format is in the following example:

In [4]:
# data preparation
Orders = pd.DataFrame({"order_id":np.arange(1,10),
             "customer_id":[1,1,2,3,4,4,5,5,5],
             "order_date":pd.to_datetime(["2020-06-01","2020-06-08","2020-06-02","2020-06-03","2020-06-04","2020-06-05",
                                         "2020-06-05","2020-06-14","2020-06-21"]),
             "item_id":[1,2,1,3,4,5,1,4,3],
             "quantity":[10,10,5,5,1,5,10,5,5]})
Items = pd.DataFrame({"item_id":np.arange(1,7),
                     "item_name":["LC Alg. Book","LC DB. Book","LC SmarthPhone","LC Phone 2020",
                                 "LC SmartGlass","LC T-Shirt XL"],
                     "item_category":["Book","Book","Phone","Phone","Glasses","T-Shirt"]})

In [5]:
Orders.head()

,order_id,customer_id,order_date,item_id,quantity
0,1,1,2020-06-01,1,10
1,2,1,2020-06-08,2,10
2,3,2,2020-06-02,1,5
3,4,3,2020-06-03,3,5
4,5,4,2020-06-04,4,1


In [6]:
Items.head()

,item_id,item_name,item_category
0,1,LC Alg. Book,Book
1,2,LC DB. Book,Book
2,3,LC SmarthPhone,Phone
3,4,LC Phone 2020,Phone
4,5,LC SmartGlass,Glasses


In [7]:
# first we need to merge the two tables using right join
merged_df = pd.merge(Orders,Items,on="item_id",how="right")
merged_df

,order_id,customer_id,order_date,item_id,quantity,item_name,item_category
0,1.0,1.0,2020-06-01,1,10.0,LC Alg. Book,Book
1,3.0,2.0,2020-06-02,1,5.0,LC Alg. Book,Book
2,7.0,5.0,2020-06-05,1,10.0,LC Alg. Book,Book
3,2.0,1.0,2020-06-08,2,10.0,LC DB. Book,Book
4,4.0,3.0,2020-06-03,3,5.0,LC SmarthPhone,Phone
5,9.0,5.0,2020-06-21,3,5.0,LC SmarthPhone,Phone
6,5.0,4.0,2020-06-04,4,1.0,LC Phone 2020,Phone
7,8.0,5.0,2020-06-14,4,5.0,LC Phone 2020,Phone
8,6.0,4.0,2020-06-05,5,5.0,LC SmartGlass,Glasses
9,NaN,NaN,NaT,6,NaN,LC T-Shirt XL,T-Shirt


In [34]:
MergedfWithoutNull


,order_id,customer_id,order_date,item_id,quantity,item_name,item_category,day_of_week
0,1.0,1.0,2020-06-01,1,10.0,LC Alg. Book,Book,Monday
1,3.0,2.0,2020-06-02,1,5.0,LC Alg. Book,Book,Tuesday
2,7.0,5.0,2020-06-05,1,10.0,LC Alg. Book,Book,Friday
3,2.0,1.0,2020-06-08,2,10.0,LC DB. Book,Book,Monday
4,4.0,3.0,2020-06-03,3,5.0,LC SmarthPhone,Phone,Wednesday
5,9.0,5.0,2020-06-21,3,5.0,LC SmarthPhone,Phone,Sunday
6,5.0,4.0,2020-06-04,4,1.0,LC Phone 2020,Phone,Thursday
7,8.0,5.0,2020-06-14,4,5.0,LC Phone 2020,Phone,Sunday
8,6.0,4.0,2020-06-05,5,5.0,LC SmartGlass,Glasses,Friday


In [39]:
from datetime import datetime

MergedfWithoutNull = merged_df.dropna()
MergedfWithoutNull["day_of_week"] = MergedfWithoutNull.order_date.apply(lambda x:datetime.strftime(x,"%A"))
MergedfWithoutNull.loc[len(MergedfWithoutNull.index)] = [np.nan,np.nan,np.nan,6,np.nan,"LC T-Shirt XL","T-Shirt",np.nan] 
#MergedfWithoutNull
MergedfWithoutNull.pivot_table(index="item_category",columns="day_of_week",values="quantity",aggfunc="sum")

/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


day_of_week,Friday,Monday,Sunday,Thursday,Tuesday,Wednesday
item_category,,,,,,
Book,10.0,20.0,NaN,NaN,5.0,NaN
Glasses,5.0,NaN,NaN,NaN,NaN,NaN
Phone,NaN,NaN,10.0,1.0,NaN,5.0


In [40]:
MergedfWithoutNull

,order_id,customer_id,order_date,item_id,quantity,item_name,item_category,day_of_week
0,1.0,1.0,2020-06-01,1,10.0,LC Alg. Book,Book,Monday
1,3.0,2.0,2020-06-02,1,5.0,LC Alg. Book,Book,Tuesday
2,7.0,5.0,2020-06-05,1,10.0,LC Alg. Book,Book,Friday
3,2.0,1.0,2020-06-08,2,10.0,LC DB. Book,Book,Monday
4,4.0,3.0,2020-06-03,3,5.0,LC SmarthPhone,Phone,Wednesday
5,9.0,5.0,2020-06-21,3,5.0,LC SmarthPhone,Phone,Sunday
6,5.0,4.0,2020-06-04,4,1.0,LC Phone 2020,Phone,Thursday
7,8.0,5.0,2020-06-14,4,5.0,LC Phone 2020,Phone,Sunday
8,6.0,4.0,2020-06-05,5,5.0,LC SmartGlass,Glasses,Friday
9,NaN,NaN,NaT,6,NaN,LC T-Shirt XL,T-Shirt,NaN
